In [1]:
using NBInclude
nbinclude("../src/PoissonProcessEstimation.ipynb")
import PoissonProcessEstimation

In [2]:
# Set up kernels
KernList = [MLKernels.GaussianKernel(400.0), MLKernels.GaussianKernel(200.0), MLKernels.GaussianKernel(100.0), MLKernels.GaussianKernel(50.0)]

4-element Array{MLKernels.ExponentialKernel{Float64,:γ1},1}:
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=400.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=200.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=100.0,γ=1.0)
 Exponential{Float64}(κ=SquaredDistance(t=1.0),α=50.0,γ=1.0) 

In [3]:
# Initialise params
θ_true = PoissonProcessEstimation.create_rand_params(20,30,length(KernList),[30,15,10,8]);
# Set the log firing rate
θ_true.b = log(20)*ones(size(θ_true.b)); 
θ_true.C = randn(size(θ_true.C))*1;

In [4]:
# Simulate from the model
data = PoissonProcessEstimation.simulate(θ_true, KernList);

Simulating trial 1
Simulating trial 2
Simulating trial 3
Simulating trial 4
Simulating trial 5
Simulating trial 6
Simulating trial 7
Simulating trial 8
Simulating trial 9
Simulating trial 10
Simulating trial 11
Simulating trial 12
Simulating trial 13
Simulating trial 14
Simulating trial 15
Simulating trial 16
Simulating trial 17
Simulating trial 18
Simulating trial 19
Simulating trial 20


In [6]:
PoissonProcessEstimation.plot_params(data, θ_true, θ_true, KernList, num_neur=1, trials=1:5, dim_latent=[])

In [8]:
θ = PoissonProcessEstimation.create_rand_params(20,30,length(KernList),[30,15,10,8]);
θ.C = randn(size(θ.C))*0.5
θ.b = θ_true.b;

In [9]:
θ_orig = θ
θ_opt = deepcopy(θ)
C, α, u, N, M, D, J = PoissonProcessEstimation.name_params(θ_opt);

In [10]:
@time Ktu, dtKtu, ddtKtu, Kuu, I = 
    PoissonProcessEstimation.optimise!(data, θ_opt, KernList, η_RKHS=1e1, η_Cm=1e1, num_iters = 5);

  3.656241 seconds (97.13 M allocations: 1.518 GB, 3.98% gc time)
Kernels have been built

Starting optimisation, initial cost is [3525.6658243307415]

Starting iter 1
  1.446619 seconds (1.29 M allocations: 99.742 MB, 1.91% gc time)
Alphas inferred, cost is [1657.312673890072]
  0.249386 seconds (273.85 k allocations: 39.663 MB, 2.12% gc time)
C learned, cost is [1975.5247172604156]

Starting iter 2
  0.099491 seconds (89.43 k allocations: 44.509 MB, 7.01% gc time)
Alphas inferred, cost is [3129.312822519285]
  0.096900 seconds (145.58 k allocations: 33.440 MB, 6.68% gc time)
C learned, cost is [3359.4079987771343]

Starting iter 3
  0.083308 seconds (89.43 k allocations: 44.509 MB, 15.12% gc time)
Alphas inferred, cost is [3492.1559681373606]
  0.101104 seconds (145.58 k allocations: 33.440 MB, 2.61% gc time)
C learned, cost is [3632.5108853884785]

Starting iter 4
  0.069971 seconds (89.43 k allocations: 44.509 MB, 9.04% gc time)
Alphas inferred, cost is [3433.0751370259086]
  0.095

In [52]:
PoissonProcessEstimation.plot_params(data, θ_opt, θ_true, KernList, num_neur=2, trials=1:5, dim_latent=[])

In [12]:
# Check for subspace angle optimisation
[PoissonProcessEstimation.subspace(θ_opt.C, θ_true.C)[1] PoissonProcessEstimation.subspace(θ_orig.C, θ_true.C)[1]]

4x2 Array{Float64,2}:
  3.97609  54.285 
  9.03587  59.1936
 15.8792   68.5126
 53.2139   81.6326

In [73]:
θ_opt.b = zeros(size(θ_opt.b))

# Check for correlation of predicted firing rates
opt_corrs = zeros(N,M)
rand_corrs = zeros(N,M)
rate_true = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_true, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
rate_est = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_opt, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
rate_rand = PoissonProcessEstimation.firing_rates(0:0.01:1, θ_orig, KernList, n_range=1:N, m_range=1:M, d_range=1:D)
for n = 1:N
    for m = 1:M
        opt_corrs[n,m] = cor(rate_true[n,m], rate_est[n,m])
        rand_corrs[n,m] = cor(rate_true[n,m], rate_rand[n,m])
    end
end

# Show the computed average correlation per neuron
[mean(opt_corrs,1)' mean(rand_corrs,1)']

30x2 Array{Float64,2}:
 0.819864  -0.0561484 
 0.780031   0.182913  
 0.69529   -0.06383   
 0.738738  -0.130192  
 0.799188  -0.0182909 
 0.77142   -0.00818592
 0.553142  -0.160032  
 0.838494   0.0242756 
 0.772248  -0.0984672 
 0.737082  -0.00905394
 0.862932   0.169713  
 0.69847    0.0592367 
 0.75268    0.0883101 
 ⋮                    
 0.756284  -0.0194006 
 0.801173   0.0149647 
 0.779802   0.0536722 
 0.566771   0.0247883 
 0.744648  -0.0641643 
 0.767389  -0.0572004 
 0.852196   0.0261047 
 0.795247   0.0489351 
 0.85313   -0.0192052 
 0.798093   0.0313124 
 0.77424   -0.0238968 
 0.852419  -0.0614187 